In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [7]:
import numpy as np
import pandas as pd

df = pd.read_csv('./kaggle/input/spaceship-titanic/train_CryoSleepNull_update.csv')
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,1.0,0.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,5.0,0.0,1.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,0.0,0.0,1.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,0.0,0.0,1.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,5.0,1.0,1.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9246_01,0.0,1.0,6.0,1490.0,1.0,2.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
8689,9259_01,0.0,0.0,5.0,1893.0,0.0,2.0,44.0,0.0,1030.0,1015.0,0.0,11.0,NaN,Annah Gilleyons,1.0
8690,9275_03,1.0,0.0,0.0,97.0,0.0,2.0,30.0,0.0,0.0,3208.0,0.0,2.0,330.0,Atlasym Conable,1.0
8691,3896_01,1.0,0.0,2.0,140.0,1.0,0.0,18.0,0.0,0.0,4387.0,0.0,2241.0,0.0,Ainoxa Preeldy,1.0


컬럼
- Destination 목적지 {55 Cancri e, PSO J318.5-22, TRAPPIST-1e}
- HomePlanet  {Earth , Europa, Mars}
- Cabin의 종류 6560
    - Cabin을 앞 대문자만 남기면 의미가 있을 수 있을까?
    - 돈이 많은 사람이 사는 자리 등으로 의미가 있는지확인
    - A,B,C,D,E,F,G,T로 변환 후 다시 숫자로 0~7까지 변환
- True / False 
    - CryoSleep
    - VIP
    - Transported(target)
- 삭제
    - PassengerId
    - Name

### null값 처리
XGBOOST를 사용해서 pridct값을 null값으로 대체

In [8]:
# df_vip =  df_train중에서 VIP의 null값이 존재하는 열만 추출 
# -> 훈련된 모델의 값으로 다시 채울 예정
df_vip = df[df['VIP'].isnull()]

#df_train 중에서 VIP의 null값이 존재하는 열만 추출 -> 추후 훈련할 데이터프레임
df_train01 = df.dropna(subset=['VIP'])
# df_train
df[df['VIP'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
38,0036_01,0.0,0.0,5.0,8.0,1.0,0.0,15.0,NaN,0.0,492.0,48.0,20.0,401.0,Marina Leodger,0.0
99,0108_03,0.0,0.0,6.0,19.0,1.0,2.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,Oline Handertiz,1.0
139,0165_01,2.0,1.0,5.0,37.0,0.0,2.0,35.0,NaN,0.0,0.0,0.0,0.0,0.0,Graven Anche,1.0
217,0244_02,2.0,1.0,5.0,47.0,1.0,0.0,14.0,NaN,0.0,0.0,0.0,0.0,0.0,Tous Sad,1.0
535,0593_01,2.0,0.0,3.0,24.0,0.0,2.0,NaN,NaN,43.0,152.0,182.0,1.0,2005.0,Hon Kra,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8533,2747_01,2.0,1.0,5.0,570.0,0.0,2.0,49.0,NaN,0.0,0.0,0.0,0.0,0.0,Stisps Chrie,1.0
8535,2822_02,0.0,0.0,6.0,450.0,1.0,2.0,5.0,NaN,0.0,0.0,0.0,0.0,NaN,Salley Harverez,0.0
8601,5777_01,0.0,0.0,5.0,1199.0,0.0,1.0,46.0,NaN,559.0,25.0,NaN,22.0,765.0,Katen River,0.0
8608,6035_02,0.0,1.0,6.0,976.0,0.0,2.0,3.0,NaN,0.0,0.0,0.0,0.0,0.0,Ginald Morsentley,0.0


In [11]:
# train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId','VIP','Name'],axis=1),df_train01['VIP'], test_size=0.2, random_state=42)

# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_train.shape
y_train.shape
X_test.shape
y_test.shape

(6792, 13)

(6792,)

(1698, 13)

(1698,)

In [12]:
#XGBoost1
import xgboost as xgb
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = xgb.XGBClassifier(early_stopping_rounds=10)
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

pred = model.predict(X_test)
acc = accuracy_score(y_test,pred)
acc

# from sklearn.metrics import confusion_matrix
# import scikitplot as skplt

# pred = model.predict(X_test)
# # 실제 , 예측
# cm = confusion_matrix(y_test, pred)
# print('confusion_matrix : \n', cm)

# skplt.metrics.plot_confusion_matrix(y_test,pred,figsize=(8,6))
# plt.show()

# cl_report = metrics.classification_report(y_test,pred)
# print('리포트:\n',cl_report)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=10, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

0.9729093050647821

In [13]:
# df_01 은 df_train 중에서 VIP의 값이 null인 경우의 데이터프레임
df_vip_null = df_vip.drop('VIP',axis=1)
df_vip_null

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
38,0036_01,0.0,0.0,5.0,8.0,1.0,0.0,15.0,0.0,492.0,48.0,20.0,401.0,Marina Leodger,0.0
99,0108_03,0.0,0.0,6.0,19.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Oline Handertiz,1.0
139,0165_01,2.0,1.0,5.0,37.0,0.0,2.0,35.0,0.0,0.0,0.0,0.0,0.0,Graven Anche,1.0
217,0244_02,2.0,1.0,5.0,47.0,1.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,Tous Sad,1.0
535,0593_01,2.0,0.0,3.0,24.0,0.0,2.0,NaN,43.0,152.0,182.0,1.0,2005.0,Hon Kra,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8533,2747_01,2.0,1.0,5.0,570.0,0.0,2.0,49.0,0.0,0.0,0.0,0.0,0.0,Stisps Chrie,1.0
8535,2822_02,0.0,0.0,6.0,450.0,1.0,2.0,5.0,0.0,0.0,0.0,0.0,NaN,Salley Harverez,0.0
8601,5777_01,0.0,0.0,5.0,1199.0,0.0,1.0,46.0,559.0,25.0,NaN,22.0,765.0,Katen River,0.0
8608,6035_02,0.0,1.0,6.0,976.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,Ginald Morsentley,0.0


In [14]:
# 훈련된 xgboost모델을 사용해서 df_vip_null의 VIP값을 예측
pred = model.predict(df_vip_null.drop(['PassengerId','Name'],axis=1))
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [15]:

# 예측된 값을 다시 df_vip으로 삽입(null값 채우기)
df_vip['VIP'] = pred
df_vip

C:\Users\shs03\AppData\Local\Temp\ipykernel_60056\1827665191.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vip['VIP'] = pred


,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
38,0036_01,0.0,0.0,5.0,8.0,1.0,0.0,15.0,0,0.0,492.0,48.0,20.0,401.0,Marina Leodger,0.0
99,0108_03,0.0,0.0,6.0,19.0,1.0,2.0,0.0,0,0.0,0.0,0.0,0.0,0.0,Oline Handertiz,1.0
139,0165_01,2.0,1.0,5.0,37.0,0.0,2.0,35.0,0,0.0,0.0,0.0,0.0,0.0,Graven Anche,1.0
217,0244_02,2.0,1.0,5.0,47.0,1.0,0.0,14.0,0,0.0,0.0,0.0,0.0,0.0,Tous Sad,1.0
535,0593_01,2.0,0.0,3.0,24.0,0.0,2.0,NaN,0,43.0,152.0,182.0,1.0,2005.0,Hon Kra,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8533,2747_01,2.0,1.0,5.0,570.0,0.0,2.0,49.0,0,0.0,0.0,0.0,0.0,0.0,Stisps Chrie,1.0
8535,2822_02,0.0,0.0,6.0,450.0,1.0,2.0,5.0,0,0.0,0.0,0.0,0.0,NaN,Salley Harverez,0.0
8601,5777_01,0.0,0.0,5.0,1199.0,0.0,1.0,46.0,0,559.0,25.0,NaN,22.0,765.0,Katen River,0.0
8608,6035_02,0.0,1.0,6.0,976.0,0.0,2.0,3.0,0,0.0,0.0,0.0,0.0,0.0,Ginald Morsentley,0.0


In [16]:

# null값을 채운 dataframe과 df_train01 합치기 
df_update = pd.concat([df_train01,df_vip],axis=0)
df_update

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,1.0,0.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,5.0,0.0,1.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,0.0,0.0,1.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,0.0,0.0,1.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,5.0,1.0,1.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8533,2747_01,2.0,1.0,5.0,570.0,0.0,2.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0,Stisps Chrie,1.0
8535,2822_02,0.0,0.0,6.0,450.0,1.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,NaN,Salley Harverez,0.0
8601,5777_01,0.0,0.0,5.0,1199.0,0.0,1.0,46.0,0.0,559.0,25.0,NaN,22.0,765.0,Katen River,0.0
8608,6035_02,0.0,1.0,6.0,976.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,Ginald Morsentley,0.0


In [17]:
df_update[df_update['VIP'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


In [18]:
df_update.to_csv('./kaggle/input/spaceship-titanic/train_VIPNull_update.csv', index=False)
